# Assignment

here comes info about the assignment

## Import libraries

here come all the imported libraries

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd

## Part I - Data import

Here comes the first part of the assignment. Importing the filtered data. Create a dataframe.
Show the data in a table.


In [12]:
file_path = 'data/who_ambient_air_quality_database_version_2023.csv'

df = pd.read_csv(file_path, sep=';')  # The file had an incorrect delimiter which caused a ParserError,
                                      # this is solved by adding sep=';' when reading the file. 

/Users/ralfhardy/opt/anaconda3/envs/TIL6022/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
df

,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
0,3_Sear,IND,India,Chennai,2018,version 2022,NaN,30.0,NaN,NaN,91.0,NaN,NaN,"U.S. Department of State, United States Enviro...",https://www.airnow.gov/index.cfm?action=airnow...,9890427,NaN,"13,08784","80,27847",1
1,3_Sear,IND,India,Solapur,2016,"version 2022, version 2018",NaN,39.0,NaN,NaN,99.0,NaN,NaN,"Central Pollution Control Board India, Environ...",NaN,985568,NaN,"17,6599188","75,9063906",1
2,3_Sear,IND,India,Chennai,2019,version 2022,NaN,39.0,NaN,NaN,85.0,NaN,NaN,"U.S. Department of State, United States Enviro...","[[[""EPA AirNow DOS"",""http://airnow.gov/index.c...",9890427,NaN,"13,08784","80,27847",1
3,3_Sear,IND,India,Hyderabad,2019,version 2022,NaN,42.0,NaN,NaN,87.0,NaN,NaN,"U.S. Department of State, United States Enviro...","[[[""EPA AirNow DOS"",""http://airnow.gov/index.c...",8943523,NaN,"17,38405","78,45636",1
4,3_Sear,IND,India,Pune,2017,version 2022,NaN,43.0,NaN,NaN,NaN,NaN,NaN,"Central Pollution Control Board India, Environ...",http://www.cpcb.gov.in/CAAQM/,5727530,NaN,"18,5053196","73,8238391",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41359,5_Emr,SAU,Saudi Arabia,Jizan,2014,version 2023,148,NaN,NaN,NaN,NaN,NaN,NaN,"Ministry of Environment, Water, and Agriculture",NaN,127743,NaN,"16,885875","42,573386",1
41360,5_Emr,SAU,Saudi Arabia,Jizan,2013,version 2023,208,NaN,NaN,NaN,NaN,NaN,NaN,"Ministry of Environment, Water, and Agriculture",NaN,127743,NaN,"16,885875","42,573386",1
41361,5_Emr,SAU,Saudi Arabia,Jizan,2012,version 2023,184,NaN,NaN,NaN,NaN,NaN,NaN,"Ministry of Environment, Water, and Agriculture",NaN,127743,NaN,"16,885875","42,573386",1
41362,5_Emr,SAU,Saudi Arabia,Jizan,2011,version 2023,316,NaN,NaN,NaN,NaN,NaN,NaN,"Ministry of Environment, Water, and Agriculture",NaN,127743,NaN,"16,885875","42,573386",1


In [15]:
df = df[df['who_region'] == '4_Eur']  #Filtered out all the non-european data
df

,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
36,4_Eur,ALB,Albania,Elbasan,2015,version 2023,NaN,NaN,24.0,NaN,NaN,98.0,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"41,111","20,0817",1
37,4_Eur,ALB,Albania,Elbasan,2016,version 2023,NaN,NaN,26.0,NaN,NaN,96.0,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"41,111","20,0817",1
38,4_Eur,ALB,Albania,Elbasan,2017,version 2023,NaN,NaN,25.0,NaN,NaN,89.0,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"41,111","20,0817",1
39,4_Eur,ALB,Albania,Elbasan,2018,version 2023,NaN,NaN,NaN,NaN,NaN,NaN,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"41,111","20,0817",1
40,4_Eur,ALB,Albania,Elbasan,2019,version 2023,NaN,NaN,NaN,NaN,NaN,NaN,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"41,111","20,0817",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41259,4_Eur,CHE,Switzerland,Jungfraujoch,2014,version 2022,2,NaN,0.0,NaN,NaN,NaN,1 Background,BAFU,http://www.arias.ch/ibonline/ib_online.php,NaN,NaN,"46,5474928","7,98507045",1
41260,4_Eur,SWE,Sweden,Markaryd,2018,version 2023,NaN,NaN,NaN,NaN,NaN,NaN,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"56,4619","13,597",1
41261,4_Eur,CHE,Switzerland,Jungfraujoch,2016,version 2022,2,NaN,0.0,NaN,NaN,NaN,1 Background,BAFU,http://www.arias.ch/ibonline/ib_online.php,NaN,NaN,"46,5474928","7,98507045",1
41262,4_Eur,TUR,Türkiye,Sirnak,2016,version 2023,1,NaN,NaN,100.0,NaN,NaN,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"37,52","42,46",1


In [ ]:
# Maybe we should merge the data so that we have one row per country (instead of now per city within a country)

## Part II - Data processing

Peak finding. Check of common peaks between data sets. 

## Part III - Data visualisation

Conclusion. Relate found info from Part II to our RQ. Perhaps with graphs, maps, etc. 